In [1]:
# Changed code relative to others:
# encoding no longer use cos sin (no noise)
# dump log now dump both pickle and txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [4]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
    return np.asarray(train_acc), np.asarray(test_acc)

In [5]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "w")
    file.write("Max train: %.2f \tMax test: %.2f \n"%(train_acc*100, test_acc*100))
    file.write(str(param))
    file.close()

In [6]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()
traindata = traindata[:20000]

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [7]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 2000, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'checkpoints': False, 'kernel': <Kernel_T.DOT: 0>, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 3, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 80, 'nFeatures': 784, 'nClasses': 10}


In [ ]:
################# VANILLA #################
hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

In [ ]:
# Retrieve info upto basis generator, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [ ]:
hde = HDE.HD_encoder(basis)

trainencoded = hde.encodeData(traindata)
#HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
#HDE.saveEncoded(testencoded, testlabels, bid, "test")

In [ ]:
# Retrieve info upto encoder, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)
#trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
#testencoded, testlabels = HDE.loadEncoded("encoded_%d_test.pkl"%bid)

In [ ]:
# Should have 90%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)
dump_log(param, max(train_acc), max(test_acc), "Baseline_"+param["id"])

In [8]:
################# BAKLAVA #################

# Some manuel tweaking 
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 2000, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'checkpoints': False, 'kernel': <Kernel_T.DOT: 0>, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 3, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 80, 'nFeatures': 784, 'nClasses': 10}


In [10]:
#param = Config.config
#param["nFeatures"] = nFeatures
#param["nClasses"] = nClasses

hdb = HDB.HD_basis(HDB.Generator_T.Baklava, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

(28, 28, 400)
(28, 28, 400)
(28, 28, 400)
(28, 28, 400)
(28, 28, 400)
3611


Configuring baklava HD basis of 5 layers:
Layer No.0: 	 400 dimension vector with kernel of size 3
Layer No.1: 	 400 dimension vector with kernel of size 3
Layer No.2: 	 400 dimension vector with kernel of size 3
Layer No.3: 	 400 dimension vector with kernel of size 3
Layer No.4: 	 400 dimension vector with kernel of size 3
Encoding time: 0.12125253677368164 


In [11]:
# Retrieve info upto basis generator, given correct bid
#bid = 2775
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [12]:
hde = HDE.HD_encoder(basis, False)

trainencoded = hde.encodeData(traindata)
#DE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
#HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (20000, 784)


Time spent: 14 sec
Encoding data of shape (10000, 784)


Time spent: 7 sec


In [13]:
# Should have 96%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.DOT: 0>)] 


Train: 0.994000 	 	 Test: 0.953200
Train: 0.999050 	 	 Test: 0.955100
Train: 0.999650 	 	 Test: 0.956500
Train: 1.000000 	 	 Test: 0.956300


In [14]:
param_to_save = dict()
param_to_save["D"] = param["D"]
param_to_save["nLayers"] = param["nLayers"]
param_to_save["uniform_dim"] = param["uniform_dim"]
param_to_save["uniform_ker"] = param["uniform_ker"]
param_to_save['dArr'] = param['dArr']
param_to_save['k'] = param['k']
param_to_save['kArr'] = param['kArr'] 
dump_log(param_to_save, max(train_acc), max(test_acc), "ConvHD_"+param["id"])

In [8]:
################# BAKLAVA, but in Suit #################

In [13]:
def Baklava_suit(param):
    
    hdb = HDB.HD_basis(HDB.Generator.Baklava, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    # Update param with bid
    param = hdb.getParam()
    print("Assigned BID: ", bid)

    hde = HDE.HD_encoder(basis, False)

    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)

    hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
    train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)

    return train_acc, test_acc

In [14]:
def extract_bak_param(param):
    param_to_save = dict()
    param_to_save["D"] = param["D"]
    param_to_save["nLayers"] = param["nLayers"]
    param_to_save["uniform_dim"] = param["uniform_dim"]
    param_to_save["uniform_ker"] = param["uniform_ker"]
    param_to_save['dArr'] = param['dArr']
    param_to_save['k'] = param['k']
    param_to_save['kArr'] = param['kArr'] 
    return param_to_save

In [43]:
# l_, r_ : min (inclusive) and max (exclusive)
# n: number of layer; k: kernel size; d: layer dimension; 
# k_dist: probability distribution, length should be r_k - l_k
# Dimension are evenly distributed towards nLayers 

# Return randomized number of layer and array of kernel size 
def Bak_config_randomizer(l_n, r_n, l_k, r_k, k_dist = None):
    nLayers = np.random.randint(r_n - l_n) + l_n
    if k_dist is None:
        kArr = np.random.randint(r_k - l_k, size = nLayers) + l_k
    else:
        kArr = np.random.choice(r_k - l_k, size = nLayers, p = k_dist) + l_k
    return nLayers, kArr
    

In [44]:
param["uniform_dim"] = 1
param['dArr'] = None
param["uniform_ker"] = 0

In [45]:
for i in range(20): 
    
    nLayers, kArr = Bak_config_randomizer(3, 10, 1, 10)
    param["nLayers"] = nLayers
    param["kArr"] = kArr
    param_to_save = extract_bak_param(param)
    print(param_to_save)
    
    train_accs = []
    test_accs = []
    for i in range(param["iter_per_trial"]):
        train_acc, test_acc = Baklava_suit(param)
        train_accs.append(max(train_acc))
        test_accs.append(max(test_acc))
    train_acc = np.mean(train_accs)
    test_acc = np.mean(test_accs)
    dump_log(param_to_save, train_acc, test_acc, "ConvHD_"+str(i))
    

{'D': 5000, 'nLayers': 4, 'uniform_dim': 1, 'uniform_ker': 0, 'dArr': None, 'k': array([8, 7, 3, 6, 7, 4, 1, 4, 3]), 'kArr': array([6, 3, 4, 5])}
(28, 28, 1250)
(28, 28, 1250)
(28, 28, 1250)
(28, 28, 1250)


Configuring baklava HD basis of 4 layers:
Layer No.0: 	 1250 dimension vector with kernel of size 6
Layer No.1: 	 1250 dimension vector with kernel of size 3
Layer No.2: 	 1250 dimension vector with kernel of size 4
Layer No.3: 	 1250 dimension vector with kernel of size 5
Encoding time: 0.20990490913391113 
Dumping basis into file: base_3591.pkl 


Assigned BID:  3591


Encoding data of shape (20000, 784)


Time spent: 30 sec
Encoding data of shape (10000, 784)


...

Encoding data of shape (20000, 784)


Time spent: 30 sec
Encoding data of shape (10000, 784)


Time spent: 16 sec


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.994650 	 	 Test: 0.965100
Train: 0.998450 	 	 Test: 0.969300
Train: 0.999700 	 	 Test: 0.971200
Train: 1.000000 	 	 Test: 0.970600
Train: 1.000000 	 	 Test: 0.970600


In [32]:
print(np.array(train_accs).shape)

(3,)


In [ ]:
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
hde_2 = HDB.HD_basis(HDB.Generator.Baklava, param)